In [1]:
%pylab inline

import pandas as pd
from scipy import stats
import statsmodels.api as sm
import matplotlib.pyplot as plt
import warnings
from itertools import product
import time
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import holidays

from sklearn import linear_model
from sklearn.model_selection import GridSearchCV

Populating the interactive namespace from numpy and matplotlib


In [2]:
from datetime import date

import holidays

# Готовим данные

объединяем таблицы за март-май

In [3]:
data = pd.read_csv('tabl_05_2016.csv', sep = ';')
data['mean'] = [sum(data.loc[i])/data.shape[1] for i in range(data.shape[0])]
data = data.loc[data['mean'] > 5]
data.drop(['mean'], axis='columns', inplace = True)

index = data.index

In [4]:
data = pd.read_csv('tabl_01_2015.csv', sep = ';')
data = data.loc[index]
data.head()

,2015.01.01 00:00,2015.01.01 01:00,2015.01.01 02:00,2015.01.01 03:00,2015.01.01 04:00,2015.01.01 05:00,2015.01.01 06:00,2015.01.01 07:00,2015.01.01 08:00,2015.01.01 09:00,...,2015.01.31 14:00,2015.01.31 15:00,2015.01.31 16:00,2015.01.31 17:00,2015.01.31 18:00,2015.01.31 19:00,2015.01.31 20:00,2015.01.31 21:00,2015.01.31 22:00,2015.01.31 23:00
1075,102.0,134.0,99.0,80.0,25.0,15.0,14.0,10.0,10.0,17.0,...,66.0,89.0,75.0,82.0,61.0,79.0,49.0,55.0,51.0,51.0
1076,115.0,162.0,179.0,116.0,54.0,32.0,28.0,24.0,37.0,38.0,...,233.0,219.0,210.0,232.0,208.0,196.0,168.0,142.0,115.0,116.0
1077,60.0,56.0,39.0,19.0,15.0,6.0,8.0,6.0,19.0,27.0,...,123.0,141.0,91.0,82.0,141.0,129.0,80.0,75.0,59.0,55.0
1125,103.0,134.0,127.0,79.0,69.0,14.0,27.0,12.0,14.0,24.0,...,74.0,66.0,58.0,97.0,114.0,153.0,105.0,103.0,99.0,105.0
1126,305.0,474.0,372.0,325.0,172.0,79.0,50.0,65.0,63.0,89.0,...,241.0,240.0,265.0,279.0,324.0,326.0,381.0,296.0,262.0,305.0


In [5]:
dat_2 = pd.read_csv('tabl_02_2015.csv', sep = ';')
dat_2 = dat_2.loc[index]
data = data.join(dat_2, how = 'outer')
data.head()

,2015.01.01 00:00,2015.01.01 01:00,2015.01.01 02:00,2015.01.01 03:00,2015.01.01 04:00,2015.01.01 05:00,2015.01.01 06:00,2015.01.01 07:00,2015.01.01 08:00,2015.01.01 09:00,...,2015.02.28 14:00,2015.02.28 15:00,2015.02.28 16:00,2015.02.28 17:00,2015.02.28 18:00,2015.02.28 19:00,2015.02.28 20:00,2015.02.28 21:00,2015.02.28 22:00,2015.02.28 23:00
1075,102.0,134.0,99.0,80.0,25.0,15.0,14.0,10.0,10.0,17.0,...,83.0,111.0,76.0,109.0,83.0,65.0,85.0,47.0,69.0,79.0
1076,115.0,162.0,179.0,116.0,54.0,32.0,28.0,24.0,37.0,38.0,...,252.0,200.0,211.0,240.0,214.0,188.0,165.0,150.0,133.0,123.0
1077,60.0,56.0,39.0,19.0,15.0,6.0,8.0,6.0,19.0,27.0,...,104.0,109.0,80.0,92.0,122.0,118.0,112.0,84.0,69.0,55.0
1125,103.0,134.0,127.0,79.0,69.0,14.0,27.0,12.0,14.0,24.0,...,91.0,85.0,72.0,81.0,102.0,143.0,96.0,81.0,85.0,98.0
1126,305.0,474.0,372.0,325.0,172.0,79.0,50.0,65.0,63.0,89.0,...,330.0,309.0,257.0,331.0,398.0,372.0,354.0,309.0,325.0,309.0


In [6]:
dat_3 = pd.read_csv('tabl_03_2015.csv', sep = ';')
dat_3 = dat_3.loc[index]
data = data.join(dat_3, how = 'outer')
data.head()

,2015.01.01 00:00,2015.01.01 01:00,2015.01.01 02:00,2015.01.01 03:00,2015.01.01 04:00,2015.01.01 05:00,2015.01.01 06:00,2015.01.01 07:00,2015.01.01 08:00,2015.01.01 09:00,...,2015.03.31 14:00,2015.03.31 15:00,2015.03.31 16:00,2015.03.31 17:00,2015.03.31 18:00,2015.03.31 19:00,2015.03.31 20:00,2015.03.31 21:00,2015.03.31 22:00,2015.03.31 23:00
1075,102.0,134.0,99.0,80.0,25.0,15.0,14.0,10.0,10.0,17.0,...,110.0,100.0,61.0,77.0,106.0,119.0,120.0,108.0,117.0,78.0
1076,115.0,162.0,179.0,116.0,54.0,32.0,28.0,24.0,37.0,38.0,...,226.0,179.0,137.0,170.0,181.0,189.0,203.0,159.0,155.0,98.0
1077,60.0,56.0,39.0,19.0,15.0,6.0,8.0,6.0,19.0,27.0,...,115.0,108.0,80.0,117.0,146.0,140.0,149.0,118.0,67.0,53.0
1125,103.0,134.0,127.0,79.0,69.0,14.0,27.0,12.0,14.0,24.0,...,97.0,83.0,73.0,103.0,150.0,133.0,146.0,155.0,130.0,81.0
1126,305.0,474.0,372.0,325.0,172.0,79.0,50.0,65.0,63.0,89.0,...,261.0,241.0,182.0,233.0,329.0,355.0,364.0,346.0,269.0,232.0


In [7]:
dat_4 = pd.read_csv('tabl_04_2015.csv', sep = ';')
dat_4 = dat_4.loc[index]
data = data.join(dat_4, how = 'outer')
data.head()

,2015.01.01 00:00,2015.01.01 01:00,2015.01.01 02:00,2015.01.01 03:00,2015.01.01 04:00,2015.01.01 05:00,2015.01.01 06:00,2015.01.01 07:00,2015.01.01 08:00,2015.01.01 09:00,...,2015.04.30 14:00,2015.04.30 15:00,2015.04.30 16:00,2015.04.30 17:00,2015.04.30 18:00,2015.04.30 19:00,2015.04.30 20:00,2015.04.30 21:00,2015.04.30 22:00,2015.04.30 23:00
1075,102.0,134.0,99.0,80.0,25.0,15.0,14.0,10.0,10.0,17.0,...,118.0,96.0,86.0,102.0,115.0,120.0,118.0,140.0,125.0,67.0
1076,115.0,162.0,179.0,116.0,54.0,32.0,28.0,24.0,37.0,38.0,...,244.0,167.0,154.0,190.0,238.0,216.0,198.0,198.0,177.0,99.0
1077,60.0,56.0,39.0,19.0,15.0,6.0,8.0,6.0,19.0,27.0,...,103.0,102.0,90.0,125.0,188.0,172.0,164.0,128.0,114.0,78.0
1125,103.0,134.0,127.0,79.0,69.0,14.0,27.0,12.0,14.0,24.0,...,124.0,85.0,66.0,128.0,138.0,165.0,167.0,153.0,124.0,82.0
1126,305.0,474.0,372.0,325.0,172.0,79.0,50.0,65.0,63.0,89.0,...,281.0,226.0,182.0,253.0,343.0,378.0,389.0,427.0,367.0,258.0


In [8]:
dat_5 = pd.read_csv('tabl_05_2015.csv', sep = ';')
dat_5 = dat_5.loc[index]
data = data.join(dat_5, how = 'outer')
data.head()

,2015.01.01 00:00,2015.01.01 01:00,2015.01.01 02:00,2015.01.01 03:00,2015.01.01 04:00,2015.01.01 05:00,2015.01.01 06:00,2015.01.01 07:00,2015.01.01 08:00,2015.01.01 09:00,...,2015.05.31 14:00,2015.05.31 15:00,2015.05.31 16:00,2015.05.31 17:00,2015.05.31 18:00,2015.05.31 19:00,2015.05.31 20:00,2015.05.31 21:00,2015.05.31 22:00,2015.05.31 23:00
1075,102.0,134.0,99.0,80.0,25.0,15.0,14.0,10.0,10.0,17.0,...,107.0,96.0,80.0,73.0,49.0,54.0,39.0,27.0,34.0,38.0
1076,115.0,162.0,179.0,116.0,54.0,32.0,28.0,24.0,37.0,38.0,...,231.0,218.0,191.0,187.0,160.0,149.0,125.0,93.0,92.0,42.0
1077,60.0,56.0,39.0,19.0,15.0,6.0,8.0,6.0,19.0,27.0,...,132.0,137.0,99.0,105.0,116.0,100.0,81.0,67.0,66.0,36.0
1125,103.0,134.0,127.0,79.0,69.0,14.0,27.0,12.0,14.0,24.0,...,88.0,76.0,53.0,61.0,72.0,41.0,57.0,34.0,52.0,31.0
1126,305.0,474.0,372.0,325.0,172.0,79.0,50.0,65.0,63.0,89.0,...,250.0,244.0,200.0,203.0,219.0,189.0,183.0,138.0,145.0,119.0


In [9]:
dat_6 = pd.read_csv('tabl_06_2015.csv', sep = ';')
dat_6 = dat_6.loc[index]
data = data.join(dat_6, how = 'outer')
data.head()

,2015.01.01 00:00,2015.01.01 01:00,2015.01.01 02:00,2015.01.01 03:00,2015.01.01 04:00,2015.01.01 05:00,2015.01.01 06:00,2015.01.01 07:00,2015.01.01 08:00,2015.01.01 09:00,...,2015.06.30 14:00,2015.06.30 15:00,2015.06.30 16:00,2015.06.30 17:00,2015.06.30 18:00,2015.06.30 19:00,2015.06.30 20:00,2015.06.30 21:00,2015.06.30 22:00,2015.06.30 23:00
1075,102.0,134.0,99.0,80.0,25.0,15.0,14.0,10.0,10.0,17.0,...,118.0,110.0,84.0,117.0,99.0,104.0,105.0,107.0,91.0,51.0
1076,115.0,162.0,179.0,116.0,54.0,32.0,28.0,24.0,37.0,38.0,...,233.0,180.0,154.0,195.0,166.0,172.0,189.0,154.0,153.0,98.0
1077,60.0,56.0,39.0,19.0,15.0,6.0,8.0,6.0,19.0,27.0,...,103.0,93.0,79.0,122.0,148.0,197.0,153.0,168.0,157.0,79.0
1125,103.0,134.0,127.0,79.0,69.0,14.0,27.0,12.0,14.0,24.0,...,69.0,83.0,76.0,89.0,84.0,137.0,109.0,119.0,126.0,82.0
1126,305.0,474.0,372.0,325.0,172.0,79.0,50.0,65.0,63.0,89.0,...,275.0,203.0,173.0,244.0,294.0,368.0,334.0,364.0,259.0,197.0


In [10]:
dat_7 = pd.read_csv('tabl_07_2015.csv', sep = ';')
dat_7 = dat_7.loc[index]
data = data.join(dat_7, how = 'outer')
data.head()

,2015.01.01 00:00,2015.01.01 01:00,2015.01.01 02:00,2015.01.01 03:00,2015.01.01 04:00,2015.01.01 05:00,2015.01.01 06:00,2015.01.01 07:00,2015.01.01 08:00,2015.01.01 09:00,...,2015.07.31 14:00,2015.07.31 15:00,2015.07.31 16:00,2015.07.31 17:00,2015.07.31 18:00,2015.07.31 19:00,2015.07.31 20:00,2015.07.31 21:00,2015.07.31 22:00,2015.07.31 23:00
1075,102.0,134.0,99.0,80.0,25.0,15.0,14.0,10.0,10.0,17.0,...,103.0,114.0,114.0,118.0,120.0,104.0,116.0,118.0,99.0,97.0
1076,115.0,162.0,179.0,116.0,54.0,32.0,28.0,24.0,37.0,38.0,...,233.0,173.0,130.0,197.0,197.0,211.0,197.0,224.0,195.0,155.0
1077,60.0,56.0,39.0,19.0,15.0,6.0,8.0,6.0,19.0,27.0,...,107.0,107.0,76.0,120.0,174.0,186.0,180.0,135.0,123.0,84.0
1125,103.0,134.0,127.0,79.0,69.0,14.0,27.0,12.0,14.0,24.0,...,104.0,87.0,91.0,84.0,141.0,146.0,131.0,124.0,136.0,100.0
1126,305.0,474.0,372.0,325.0,172.0,79.0,50.0,65.0,63.0,89.0,...,245.0,232.0,195.0,261.0,330.0,346.0,349.0,333.0,311.0,311.0


In [11]:
dat_8 = pd.read_csv('tabl_08_2015.csv', sep = ';')
dat_8 = dat_8.loc[index]
data = data.join(dat_8, how = 'outer')
data.head()

,2015.01.01 00:00,2015.01.01 01:00,2015.01.01 02:00,2015.01.01 03:00,2015.01.01 04:00,2015.01.01 05:00,2015.01.01 06:00,2015.01.01 07:00,2015.01.01 08:00,2015.01.01 09:00,...,2015.08.31 14:00,2015.08.31 15:00,2015.08.31 16:00,2015.08.31 17:00,2015.08.31 18:00,2015.08.31 19:00,2015.08.31 20:00,2015.08.31 21:00,2015.08.31 22:00,2015.08.31 23:00
1075,102.0,134.0,99.0,80.0,25.0,15.0,14.0,10.0,10.0,17.0,...,92.0,105.0,76.0,94.0,114.0,92.0,89.0,84.0,67.0,32.0
1076,115.0,162.0,179.0,116.0,54.0,32.0,28.0,24.0,37.0,38.0,...,189.0,152.0,132.0,161.0,147.0,187.0,181.0,152.0,113.0,81.0
1077,60.0,56.0,39.0,19.0,15.0,6.0,8.0,6.0,19.0,27.0,...,102.0,84.0,79.0,111.0,158.0,138.0,119.0,88.0,49.0,39.0
1125,103.0,134.0,127.0,79.0,69.0,14.0,27.0,12.0,14.0,24.0,...,54.0,72.0,73.0,94.0,113.0,101.0,126.0,89.0,74.0,55.0
1126,305.0,474.0,372.0,325.0,172.0,79.0,50.0,65.0,63.0,89.0,...,237.0,245.0,217.0,234.0,289.0,322.0,278.0,234.0,191.0,168.0


In [12]:
dat_9 = pd.read_csv('tabl_09_2015.csv', sep = ';')
dat_9 = dat_9.loc[index]
data = data.join(dat_9, how = 'outer')
data.head()

,2015.01.01 00:00,2015.01.01 01:00,2015.01.01 02:00,2015.01.01 03:00,2015.01.01 04:00,2015.01.01 05:00,2015.01.01 06:00,2015.01.01 07:00,2015.01.01 08:00,2015.01.01 09:00,...,2015.09.30 14:00,2015.09.30 15:00,2015.09.30 16:00,2015.09.30 17:00,2015.09.30 18:00,2015.09.30 19:00,2015.09.30 20:00,2015.09.30 21:00,2015.09.30 22:00,2015.09.30 23:00
1075,102.0,134.0,99.0,80.0,25.0,15.0,14.0,10.0,10.0,17.0,...,98.0,82.0,80.0,79.0,87.0,102.0,109.0,112.0,69.0,54.0
1076,115.0,162.0,179.0,116.0,54.0,32.0,28.0,24.0,37.0,38.0,...,160.0,158.0,125.0,152.0,144.0,189.0,208.0,237.0,194.0,111.0
1077,60.0,56.0,39.0,19.0,15.0,6.0,8.0,6.0,19.0,27.0,...,139.0,107.0,103.0,114.0,133.0,142.0,139.0,113.0,119.0,64.0
1125,103.0,134.0,127.0,79.0,69.0,14.0,27.0,12.0,14.0,24.0,...,91.0,82.0,69.0,113.0,130.0,119.0,131.0,146.0,81.0,83.0
1126,305.0,474.0,372.0,325.0,172.0,79.0,50.0,65.0,63.0,89.0,...,220.0,199.0,198.0,249.0,304.0,369.0,368.0,333.0,233.0,184.0


In [13]:
dat_10 = pd.read_csv('tabl_10_2015.csv', sep = ';')
dat_10 = dat_10.loc[index]
data = data.join(dat_10, how = 'outer')
data.head()

,2015.01.01 00:00,2015.01.01 01:00,2015.01.01 02:00,2015.01.01 03:00,2015.01.01 04:00,2015.01.01 05:00,2015.01.01 06:00,2015.01.01 07:00,2015.01.01 08:00,2015.01.01 09:00,...,2015.10.31 14:00,2015.10.31 15:00,2015.10.31 16:00,2015.10.31 17:00,2015.10.31 18:00,2015.10.31 19:00,2015.10.31 20:00,2015.10.31 21:00,2015.10.31 22:00,2015.10.31 23:00
1075,102.0,134.0,99.0,80.0,25.0,15.0,14.0,10.0,10.0,17.0,...,123.0,110.0,97.0,86.0,97.0,57.0,59.0,88.0,67.0,64.0
1076,115.0,162.0,179.0,116.0,54.0,32.0,28.0,24.0,37.0,38.0,...,264.0,237.0,216.0,251.0,242.0,219.0,186.0,146.0,161.0,154.0
1077,60.0,56.0,39.0,19.0,15.0,6.0,8.0,6.0,19.0,27.0,...,142.0,139.0,111.0,102.0,130.0,125.0,87.0,91.0,63.0,54.0
1125,103.0,134.0,127.0,79.0,69.0,14.0,27.0,12.0,14.0,24.0,...,68.0,74.0,67.0,77.0,92.0,97.0,93.0,99.0,92.0,90.0
1126,305.0,474.0,372.0,325.0,172.0,79.0,50.0,65.0,63.0,89.0,...,271.0,269.0,224.0,256.0,291.0,279.0,320.0,263.0,316.0,311.0


In [14]:
dat_11 = pd.read_csv('tabl_11_2015.csv', sep = ';')
dat_11 = dat_11.loc[index]
data = data.join(dat_11, how = 'outer')
data.head()

,2015.01.01 00:00,2015.01.01 01:00,2015.01.01 02:00,2015.01.01 03:00,2015.01.01 04:00,2015.01.01 05:00,2015.01.01 06:00,2015.01.01 07:00,2015.01.01 08:00,2015.01.01 09:00,...,2015.11.30 14:00,2015.11.30 15:00,2015.11.30 16:00,2015.11.30 17:00,2015.11.30 18:00,2015.11.30 19:00,2015.11.30 20:00,2015.11.30 21:00,2015.11.30 22:00,2015.11.30 23:00
1075,102.0,134.0,99.0,80.0,25.0,15.0,14.0,10.0,10.0,17.0,...,90.0,89.0,60.0,70.0,97.0,81.0,121.0,80.0,54.0,29.0
1076,115.0,162.0,179.0,116.0,54.0,32.0,28.0,24.0,37.0,38.0,...,195.0,190.0,162.0,185.0,147.0,183.0,151.0,151.0,138.0,92.0
1077,60.0,56.0,39.0,19.0,15.0,6.0,8.0,6.0,19.0,27.0,...,81.0,71.0,58.0,99.0,155.0,158.0,136.0,111.0,78.0,25.0
1125,103.0,134.0,127.0,79.0,69.0,14.0,27.0,12.0,14.0,24.0,...,62.0,71.0,55.0,107.0,108.0,93.0,82.0,96.0,103.0,91.0
1126,305.0,474.0,372.0,325.0,172.0,79.0,50.0,65.0,63.0,89.0,...,210.0,198.0,206.0,232.0,302.0,286.0,297.0,225.0,160.0,112.0


In [15]:
dat_12 = pd.read_csv('tabl_12_2015.csv', sep = ';')
dat_12 = dat_12.loc[index]
data = data.join(dat_12, how = 'outer')
data.head()

,2015.01.01 00:00,2015.01.01 01:00,2015.01.01 02:00,2015.01.01 03:00,2015.01.01 04:00,2015.01.01 05:00,2015.01.01 06:00,2015.01.01 07:00,2015.01.01 08:00,2015.01.01 09:00,...,2015.12.31 14:00,2015.12.31 15:00,2015.12.31 16:00,2015.12.31 17:00,2015.12.31 18:00,2015.12.31 19:00,2015.12.31 20:00,2015.12.31 21:00,2015.12.31 22:00,2015.12.31 23:00
1075,102.0,134.0,99.0,80.0,25.0,15.0,14.0,10.0,10.0,17.0,...,115.0,144.0,105.0,84.0,74.0,70.0,82.0,82.0,71.0,48.0
1076,115.0,162.0,179.0,116.0,54.0,32.0,28.0,24.0,37.0,38.0,...,280.0,248.0,189.0,189.0,166.0,206.0,196.0,197.0,128.0,103.0
1077,60.0,56.0,39.0,19.0,15.0,6.0,8.0,6.0,19.0,27.0,...,96.0,110.0,95.0,77.0,92.0,104.0,104.0,97.0,52.0,40.0
1125,103.0,134.0,127.0,79.0,69.0,14.0,27.0,12.0,14.0,24.0,...,97.0,85.0,65.0,65.0,56.0,98.0,92.0,89.0,101.0,54.0
1126,305.0,474.0,372.0,325.0,172.0,79.0,50.0,65.0,63.0,89.0,...,237.0,228.0,241.0,266.0,281.0,289.0,331.0,328.0,285.0,186.0


In [16]:
data_1 = pd.read_csv('tabl_01_2016.csv', sep = ';')
data_1 = data_1.loc[index]
data = data.join(data_1)
data.head()

,2015.01.01 00:00,2015.01.01 01:00,2015.01.01 02:00,2015.01.01 03:00,2015.01.01 04:00,2015.01.01 05:00,2015.01.01 06:00,2015.01.01 07:00,2015.01.01 08:00,2015.01.01 09:00,...,2016.01.31 14:00,2016.01.31 15:00,2016.01.31 16:00,2016.01.31 17:00,2016.01.31 18:00,2016.01.31 19:00,2016.01.31 20:00,2016.01.31 21:00,2016.01.31 22:00,2016.01.31 23:00
1075,102.0,134.0,99.0,80.0,25.0,15.0,14.0,10.0,10.0,17.0,...,60.0,61.0,68.0,64.0,54.0,40.0,48.0,29.0,27.0,30.0
1076,115.0,162.0,179.0,116.0,54.0,32.0,28.0,24.0,37.0,38.0,...,201.0,160.0,159.0,163.0,145.0,95.0,70.0,45.0,42.0,30.0
1077,60.0,56.0,39.0,19.0,15.0,6.0,8.0,6.0,19.0,27.0,...,113.0,100.0,88.0,107.0,102.0,71.0,45.0,46.0,27.0,13.0
1125,103.0,134.0,127.0,79.0,69.0,14.0,27.0,12.0,14.0,24.0,...,54.0,72.0,62.0,71.0,64.0,52.0,36.0,31.0,22.0,24.0
1126,305.0,474.0,372.0,325.0,172.0,79.0,50.0,65.0,63.0,89.0,...,217.0,197.0,208.0,213.0,204.0,182.0,125.0,82.0,82.0,63.0


In [17]:
data_2 = pd.read_csv('tabl_02_2016.csv', sep = ';')
data_2 = data_2.loc[data.index]
data_2 = data_2[data_2.columns[:-24]]
data = data.join(data_2)
data.head()

,2015.01.01 00:00,2015.01.01 01:00,2015.01.01 02:00,2015.01.01 03:00,2015.01.01 04:00,2015.01.01 05:00,2015.01.01 06:00,2015.01.01 07:00,2015.01.01 08:00,2015.01.01 09:00,...,2016.02.28 14:00,2016.02.28 15:00,2016.02.28 16:00,2016.02.28 17:00,2016.02.28 18:00,2016.02.28 19:00,2016.02.28 20:00,2016.02.28 21:00,2016.02.28 22:00,2016.02.28 23:00
1075,102.0,134.0,99.0,80.0,25.0,15.0,14.0,10.0,10.0,17.0,...,97.0,112.0,98.0,98.0,92.0,68.0,38.0,35.0,32.0,41.0
1076,115.0,162.0,179.0,116.0,54.0,32.0,28.0,24.0,37.0,38.0,...,216.0,213.0,208.0,193.0,193.0,143.0,80.0,57.0,54.0,44.0
1077,60.0,56.0,39.0,19.0,15.0,6.0,8.0,6.0,19.0,27.0,...,112.0,110.0,100.0,101.0,103.0,100.0,40.0,51.0,29.0,19.0
1125,103.0,134.0,127.0,79.0,69.0,14.0,27.0,12.0,14.0,24.0,...,80.0,76.0,57.0,67.0,77.0,62.0,28.0,26.0,26.0,21.0
1126,305.0,474.0,372.0,325.0,172.0,79.0,50.0,65.0,63.0,89.0,...,291.0,216.0,222.0,237.0,250.0,186.0,136.0,89.0,91.0,72.0


In [18]:
data_3 = pd.read_csv('tabl_03_2016.csv', sep = ';')
data_3 = data_3.loc[data.index]
data = data.join(data_3)
data.head()

,2015.01.01 00:00,2015.01.01 01:00,2015.01.01 02:00,2015.01.01 03:00,2015.01.01 04:00,2015.01.01 05:00,2015.01.01 06:00,2015.01.01 07:00,2015.01.01 08:00,2015.01.01 09:00,...,2016.03.31 14:00,2016.03.31 15:00,2016.03.31 16:00,2016.03.31 17:00,2016.03.31 18:00,2016.03.31 19:00,2016.03.31 20:00,2016.03.31 21:00,2016.03.31 22:00,2016.03.31 23:00
1075,102.0,134.0,99.0,80.0,25.0,15.0,14.0,10.0,10.0,17.0,...,122.0,105.0,73.0,98.0,97.0,117.0,107.0,160.0,111.0,77.0
1076,115.0,162.0,179.0,116.0,54.0,32.0,28.0,24.0,37.0,38.0,...,246.0,200.0,162.0,182.0,211.0,203.0,203.0,190.0,151.0,99.0
1077,60.0,56.0,39.0,19.0,15.0,6.0,8.0,6.0,19.0,27.0,...,134.0,94.0,82.0,117.0,139.0,177.0,173.0,104.0,106.0,45.0
1125,103.0,134.0,127.0,79.0,69.0,14.0,27.0,12.0,14.0,24.0,...,108.0,98.0,73.0,124.0,159.0,150.0,160.0,147.0,143.0,110.0
1126,305.0,474.0,372.0,325.0,172.0,79.0,50.0,65.0,63.0,89.0,...,253.0,253.0,196.0,289.0,374.0,369.0,376.0,342.0,311.0,306.0


In [19]:
data_4 = pd.read_csv('tabl_04_2016.csv', sep = ';')
data_4 = data_4.loc[data.index]
data = data.join(data_4)
data.head()

,2015.01.01 00:00,2015.01.01 01:00,2015.01.01 02:00,2015.01.01 03:00,2015.01.01 04:00,2015.01.01 05:00,2015.01.01 06:00,2015.01.01 07:00,2015.01.01 08:00,2015.01.01 09:00,...,2016.04.30 14:00,2016.04.30 15:00,2016.04.30 16:00,2016.04.30 17:00,2016.04.30 18:00,2016.04.30 19:00,2016.04.30 20:00,2016.04.30 21:00,2016.04.30 22:00,2016.04.30 23:00
1075,102.0,134.0,99.0,80.0,25.0,15.0,14.0,10.0,10.0,17.0,...,138.0,145.0,162.0,142.0,157.0,121.0,107.0,91.0,100.0,92.0
1076,115.0,162.0,179.0,116.0,54.0,32.0,28.0,24.0,37.0,38.0,...,318.0,299.0,214.0,234.0,223.0,209.0,208.0,155.0,97.0,118.0
1077,60.0,56.0,39.0,19.0,15.0,6.0,8.0,6.0,19.0,27.0,...,144.0,147.0,112.0,111.0,124.0,145.0,99.0,108.0,85.0,63.0
1125,103.0,134.0,127.0,79.0,69.0,14.0,27.0,12.0,14.0,24.0,...,91.0,91.0,75.0,97.0,111.0,101.0,107.0,115.0,100.0,86.0
1126,305.0,474.0,372.0,325.0,172.0,79.0,50.0,65.0,63.0,89.0,...,289.0,287.0,264.0,332.0,339.0,345.0,290.0,318.0,217.0,284.0


In [20]:
data_5 = pd.read_csv('tabl_05_2016.csv', sep = ';')
data_5 = data_5.loc[data.index]
data = data.join(data_5)
data.head()

,2015.01.01 00:00,2015.01.01 01:00,2015.01.01 02:00,2015.01.01 03:00,2015.01.01 04:00,2015.01.01 05:00,2015.01.01 06:00,2015.01.01 07:00,2015.01.01 08:00,2015.01.01 09:00,...,2016.05.31 14:00,2016.05.31 15:00,2016.05.31 16:00,2016.05.31 17:00,2016.05.31 18:00,2016.05.31 19:00,2016.05.31 20:00,2016.05.31 21:00,2016.05.31 22:00,2016.05.31 23:00
1075,102.0,134.0,99.0,80.0,25.0,15.0,14.0,10.0,10.0,17.0,...,124.0,105.0,80.0,84.0,106.0,106.0,99.0,84.0,83.0,34.0
1076,115.0,162.0,179.0,116.0,54.0,32.0,28.0,24.0,37.0,38.0,...,214.0,208.0,156.0,202.0,213.0,167.0,166.0,173.0,98.0,77.0
1077,60.0,56.0,39.0,19.0,15.0,6.0,8.0,6.0,19.0,27.0,...,104.0,69.0,67.0,109.0,173.0,171.0,165.0,107.0,127.0,58.0
1125,103.0,134.0,127.0,79.0,69.0,14.0,27.0,12.0,14.0,24.0,...,96.0,70.0,68.0,109.0,126.0,88.0,65.0,78.0,94.0,53.0
1126,305.0,474.0,372.0,325.0,172.0,79.0,50.0,65.0,63.0,89.0,...,205.0,213.0,168.0,250.0,281.0,293.0,301.0,256.0,174.0,118.0


In [21]:
data_6 = pd.read_csv('tabl_06_2016.csv', sep = ';')
data_6 = data_6.loc[data.index]
data = data.join(data_6)
data.head()

,2015.01.01 00:00,2015.01.01 01:00,2015.01.01 02:00,2015.01.01 03:00,2015.01.01 04:00,2015.01.01 05:00,2015.01.01 06:00,2015.01.01 07:00,2015.01.01 08:00,2015.01.01 09:00,...,2016.06.30 14:00,2016.06.30 15:00,2016.06.30 16:00,2016.06.30 17:00,2016.06.30 18:00,2016.06.30 19:00,2016.06.30 20:00,2016.06.30 21:00,2016.06.30 22:00,2016.06.30 23:00
1075,102.0,134.0,99.0,80.0,25.0,15.0,14.0,10.0,10.0,17.0,...,109.0,86.0,81.0,82.0,88.0,116.0,104.0,151.0,106.0,85.0
1076,115.0,162.0,179.0,116.0,54.0,32.0,28.0,24.0,37.0,38.0,...,236.0,199.0,152.0,139.0,173.0,190.0,142.0,162.0,168.0,130.0
1077,60.0,56.0,39.0,19.0,15.0,6.0,8.0,6.0,19.0,27.0,...,103.0,68.0,85.0,100.0,117.0,135.0,149.0,145.0,103.0,86.0
1125,103.0,134.0,127.0,79.0,69.0,14.0,27.0,12.0,14.0,24.0,...,107.0,78.0,86.0,80.0,75.0,132.0,141.0,135.0,125.0,113.0
1126,305.0,474.0,372.0,325.0,172.0,79.0,50.0,65.0,63.0,89.0,...,212.0,213.0,179.0,255.0,309.0,395.0,333.0,359.0,317.0,256.0


In [22]:
index = data.index

In [23]:
K = 23
def pred(data, K):
    year_period = 8766 #365.2424*24 - средняя продолжительность года в часах
    week_period = 168
    day_period = 24
    for i in range(1, K+1):
        data['y_с_' + str(i)] = np.cos(np.arange(1, data.shape[0]+1)*2*np.pi*i/year_period)
        data['y_s_' + str(i)] = np.sin(np.arange(1, data.shape[0]+1)*2*np.pi*i/year_period)
    for i in range(1, K+1):    
        data['w_с_' + str(i)] = np.cos(np.arange(1, data.shape[0]+1)*2*np.pi*i/week_period)
        data['w_s_' + str(i)] = np.sin(np.arange(1, data.shape[0]+1)*2*np.pi*i/week_period)
    for i in range(1, K+1):    
        data['d_с_' + str(i)] = np.cos(np.arange(1, data.shape[0]+1)*2*np.pi*i/day_period)
        data['d_s_' + str(i)] = np.sin(np.arange(1, data.shape[0]+1)*2*np.pi*i/day_period)

# Добавляем признаки

In [24]:
def add(data, i):
    data_copy = pd.DataFrame(data.loc[i])
    data_copy.columns = ['trip'] #количество поездок
    data_copy['region'] = [str(i)]*data.shape[1] #регион
    hours = [datetime.datetime.strptime(i, '%Y.%m.%d %H:%M').hour for i in data.columns] 
    data_copy['hour'] = hours                                                          #час
    month = [datetime.datetime.strptime(i, '%Y.%m.%d %H:%M').month for i in data.columns]
    data_copy['month'] = month                                                        #месяц
    days = [datetime.datetime.strptime(i, '%Y.%m.%d %H:%M').day for i in data.columns] 
    data_copy['days'] = days                            #число
    year = [datetime.datetime.strptime(i, '%Y.%m.%d %H:%M').year for i in data.columns] 
    data_copy['year'] = year  
    dayofweek = [datetime.datetime.strptime(i, '%Y.%m.%d %H:%M').weekday() for i in data.columns]
    data_copy['dayofweek'] = dayofweek                                               #день недели
    
    weekends = []

    
    for j in range(len(data_copy)):
        if data_copy['dayofweek'][j] == '1' or data_copy['dayofweek'][j] == '7':
            weekends.append(1)
        else:
            weekends.append(0)

    data_copy['weekend'] = weekends # является ли день выходным
    data_copy['y-1'] = data_copy.trip.shift(1) #количество поездок за предыдущие n часов (1, 2, 3, 4, 5, 6, 24, 48)
    data_copy['y-2'] = data_copy.trip.shift(2)
    data_copy['y-3'] = data_copy.trip.shift(3)
    data_copy['y-7'] = data_copy.trip.shift(7)
    data_copy['y-8'] = data_copy.trip.shift(8)
    data_copy['y-9'] = data_copy.trip.shift(9)
    
    data_copy['y-24'] = data_copy.trip.shift(24)
    data_copy['y-48'] = data_copy.trip.shift(48)
    
    pred(data_copy, K)
    return data_copy.dropna()

# Настраиваем модель

In [25]:
%%time
models = {}
y_dict = {}
warnings.filterwarnings('ignore')

X_train = pd.DataFrame()
y_train = pd.DataFrame()

X_test= pd.DataFrame()
y_test = pd.DataFrame()

X_pred = pd.DataFrame()
y_pred = pd.DataFrame()

for i in index:
    data_copy = add(data, i)
    
    train_stop = '2016.05.01 00:00'
    test_stop = '2016.06.01 00:00'
    
    print(i, end = ' ')
    
    X_train = pd.concat([X_train, data_copy[data_copy.columns[1:]].loc[:train_stop]], ignore_index = True)
    y_train = pd.concat([y_train, data_copy[data_copy.columns[:1]].loc[:train_stop]], ignore_index = True)
    
    X_test = pd.concat([X_test, data_copy[data_copy.columns[1:]].loc[train_stop: test_stop]], ignore_index = True)
    y_test = pd.concat([y_test, data_copy[data_copy.columns[:1]].loc[train_stop: test_stop]], ignore_index = True)
    
    X_pred = pd.concat([X_pred, data_copy[data_copy.columns[1:]].loc[test_stop:]], ignore_index = True)
    y_pred = pd.concat([y_pred, data_copy[data_copy.columns[:1]].loc[test_stop:]], ignore_index = True)
    
mse_min = mean_squared_error(y_test, [0]*len(y_test))
for alpha in np.logspace(-2, 3, 5):
    print(alpha, end = ' ')
    model = linear_model.Lasso(alpha = alpha)
    model.fit(X_train, y_train)
    mse = mean_squared_error(model.predict(X_test), y_test)
    if mse < mse_min:
        best_model = model
        mse_min = mse
        
y = best_model.predict(X_pred)


1075 1076 1077 1125 1126 1127 1128 1129 1130 1131 1132 1172 1173 1174 1175 1176 1177 1178 1179 1180 1181 1182 1183 1184 1221 1222 1223 1224 1225 1227 1228 1229 1230 1231 1232 1233 1234 1235 1272 1273 1274 1278 1279 1280 1281 1282 1283 1284 1285 1286 1287 1326 1327 1331 1332 1333 1334 1335 1336 1337 1338 1339 1376 1377 1378 1380 1382 1383 1384 1385 1386 1387 1388 1389 1390 1426 1431 1434 1435 1436 1437 1438 1439 1441 1442 1480 1482 1483 1530 1532 1533 1580 1630 1684 1733 1734 1783 2068 2069 2118 2119 2168 0.01 0.1778279410038923 3.1622776601683795 56.23413251903491 1000.0 Wall time: 24min 10s


# Считаем ошибку, создаем файл для соревнования

In [26]:
y_pred['region'] = X_pred['region']
y_pred['pred'] = y

In [28]:
s = 0
ys = []
for i in index:
    data_reg = pd.DataFrame(y_pred.loc[y_pred['region'] == str(i)])
    data_reg.index = list(range(720))
    for j in range(720-5):
        for k in range(0, 6):
            s += abs(data_reg['trip'].loc[j+k] - data_reg['pred'].loc[j+k])
            ys.append(data_reg['pred'].loc[j+k])
            #ind = str(i) + '_2016-' + t[3:5] + '-' + t[0:2] + '_' + str(int(t[6:8])) + '_' + str(i+1)
            #ys.append(y_dict[i][j+k])
            
    print(i, end = ' ')

1075 1076 1077 1125 1126 1127 1128 1129 1130 1131 1132 1172 1173 1174 1175 1176 1177 1178 1179 1180 1181 1182 1183 1184 1221 1222 1223 1224 1225 1227 1228 1229 1230 1231 1232 1233 1234 1235 1272 1273 1274 1278 1279 1280 1281 1282 1283 1284 1285 1286 1287 1326 1327 1331 1332 1333 1334 1335 1336 1337 1338 1339 1376 1377 1378 1380 1382 1383 1384 1385 1386 1387 1388 1389 1390 1426 1431 1434 1435 1436 1437 1438 1439 1441 1442 1480 1482 1483 1530 1532 1533 1580 1630 1684 1733 1734 1783 2068 2069 2118 2119 2168 

In [29]:
s/739/6/102

25.22928460028935

In [30]:
a = pd.read_csv('a.csv', sep = ',')
a['y'] = ys

In [31]:
a.to_csv('a_3.csv', sep = ',', index = False)

<img src = '2020-12-16_22-36-48.png'>

## Попробуем настраивать свою модель для каждого региона

In [32]:
%%time
models = {}
y_dict = {}
warnings.filterwarnings('ignore')

for i in index:
    data_copy = add(data, i)
    
    train_stop = '2016.05.01 00:00'
    test_stop = '2016.06.01 00:00'
    
    print(i, end = ' ')
    
    X_train = data_copy[data_copy.columns[1:]].loc[:train_stop]
    y_train = data_copy[data_copy.columns[:1]].loc[:train_stop]
    
    X_test = data_copy[data_copy.columns[1:]].loc[train_stop: test_stop]
    y_test = data_copy[data_copy.columns[:1]].loc[train_stop: test_stop]
    
    X_pred = data_copy[data_copy.columns[1:]].loc[test_stop:]
    y_pred =data_copy[data_copy.columns[:1]].loc[test_stop:]
    
    mse_min = mean_squared_error(y_test, [0]*len(y_test))
    for alpha in np.logspace(-2, 3, 10):
        model = linear_model.Lasso(alpha = alpha)
        model.fit(X_train, y_train)
        mse = mean_squared_error(model.predict(X_test), y_test)
        if mse < mse_min:
            best_model = model
            mse_min = mse
        
    models[i] = best_model
    y = best_model.predict(X_pred)
    y_dict[i] = y

1075 1076 1077 1125 1126 1127 1128 1129 1130 1131 1132 1172 1173 1174 1175 1176 1177 1178 1179 1180 1181 1182 1183 1184 1221 1222 1223 1224 1225 1227 1228 1229 1230 1231 1232 1233 1234 1235 1272 1273 1274 1278 1279 1280 1281 1282 1283 1284 1285 1286 1287 1326 1327 1331 1332 1333 1334 1335 1336 1337 1338 1339 1376 1377 1378 1380 1382 1383 1384 1385 1386 1387 1388 1389 1390 1426 1431 1434 1435 1436 1437 1438 1439 1441 1442 1480 1482 1483 1530 1532 1533 1580 1630 1684 1733 1734 1783 2068 2069 2118 2119 2168 Wall time: 50min 18s


In [38]:
s = 0
ys = []
for i in index:
    for j in range(data_6.shape[1]-5):
        for k in range(0, 6):
            s += abs(y_dict[i][j+k] - data_6.loc[i][j+k])
            #ind = str(i) + '_2016-' + t[3:5] + '-' + t[0:2] + '_' + str(int(t[6:8])) + '_' + str(i+1)
            ys.append(y_dict[i][j+k])
            
    print(i, end = ' ')

1075 1076 1077 1125 1126 1127 1128 1129 1130 1131 1132 1172 1173 1174 1175 1176 1177 1178 1179 1180 1181 1182 1183 1184 1221 1222 1223 1224 1225 1227 1228 1229 1230 1231 1232 1233 1234 1235 1272 1273 1274 1278 1279 1280 1281 1282 1283 1284 1285 1286 1287 1326 1327 1331 1332 1333 1334 1335 1336 1337 1338 1339 1376 1377 1378 1380 1382 1383 1384 1385 1386 1387 1388 1389 1390 1426 1431 1434 1435 1436 1437 1438 1439 1441 1442 1480 1482 1483 1530 1532 1533 1580 1630 1684 1733 1734 1783 2068 2069 2118 2119 2168 

In [39]:
s/739/6/102

17.958399755739435

In [35]:
a['y'] = ys

In [36]:
a.to_csv('a_4.csv', sep = ',', index = False)

<img src = '2020-12-16_22-39-55.png'>

# Выберем второй вариант